# Persistence Images in Classification

This notebook shows how you can use persistent homology and persistence images to classify datasets.  We construct datasets from two classes, one just noise and the other noise with a big circle in the middle. We then compute persistence diagrams with [Ripser.py](https://github.com/scikit-tda/ripser.py) and convert them to persistence images with [PersIm](https://github.com/scikit-tda/persim). Using these persistence images, we build a Logistic Regression model using a LASSO penatly to classify whether the dataset has a circle or not.  We find, using only default values, classification has a mean accuracy greater than 90.

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersImage

## Construct data

Generate N datasets that are just noise and N that are noise with a circle. 

In [ ]:
N = 200
N_per_class = int(N / 2)
N_in_class = 400

def noise(N, scale):
    return scale * np.random.random((N, 2))

def circle(N, scale, offset):
    return offset + scale * datasets.make_circles(n_samples=N, factor=0.4, noise=0.05)[0]
    
just_noise = [noise(N_in_class, 150) for _ in range(N_per_class)]

half = int(N_in_class / 2)
with_circle = [np.concatenate((circle(half, 50, 70), noise(half, 150)))
               for _ in range(N_per_class)]

datas = []
datas.extend(just_noise)
datas.extend(with_circle)

# Define labels
labels = np.zeros(N)
labels[N_per_class:] = 1

In [ ]:
# Visualize the data
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(10,5)

xs, ys = just_noise[0][:,0], just_noise[0][:,1]
axs[0].scatter(xs, ys)
axs[0].set_title("Example noise dataset")
axs[0].set_aspect('equal', 'box')

xs_, ys_ = with_circle[0][:,0], with_circle[0][:,1]
axs[1].scatter(xs_, ys_)
axs[1].set_title("Example noise with circle dataset")
axs[1].set_aspect('equal', 'box')

fig.tight_layout()

## Compute homology of each dataset

Generate the persistence diagram of $H_1$ for each of the datasets generated above.

In [ ]:
rips = Rips(maxdim=1, coeff=2)
diagrams = [rips.fit_transform(data) for data in datas]
diagrams_h1 = [rips.fit_transform(data)[1] for data in datas]

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)

rips.plot(diagrams_h1[0], show=False)
plt.title("PD of $H_1$ for just noise")

plt.subplot(122)
rips.plot(diagrams_h1[-1], show=False)
plt.title("PD of $H_1$ for circle w/ noise")

plt.show()

## Compute persistence images

Convert each persistence diagram into a persistence image. Flatten each image into a vector format

In [ ]:
pim = PersImage(pixels=[20,20], spread=1)
imgs = pim.transform(diagrams_h1)

In [ ]:
imgs_array = np.array([img.flatten() for img in imgs])

In [ ]:
plt.figure(figsize=(15,7.5))

for i in range(4):
    ax = plt.subplot(240+i+1)
    pim.show(imgs[i], ax)
    plt.title("PI of $H_1$ for noise")

for i in range(4):
    ax = plt.subplot(240+i+5)
    pim.show(imgs[-(i+1)], ax)
    plt.title("PI of $H_1$ for circle w/ noise")

## Classify the datasets from the persistence images

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imgs_array, labels, test_size=0.40, random_state=42)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

So these are perfectly descriminative? Seems almost too good to be true. Let's see what features are important.

## Inverse analysis on LASSO 

Visualizing the regression coefficients as a persistence image shows us which features of the images are most important for classification.

In [ ]:
inverse_image = np.copy(lr.coef_).reshape((20,20))
pim.show(inverse_image)

# Slightly less trivial: classify shapes

In [ ]:
import os
pd_data = {}
data_dir = '../data/ToyData_PD_TextFiles/'

for f in os.listdir(data_dir):
    pd_data[f] = np.genfromtxt(data_dir + f);
    

In [ ]:
dim = 1
dgms = []
labels = []
for f in pd_data:
    args = f.split('_');
    if args[-1][0] == str(dim):
        dgms.append(pd_data[f])
        labels.append(args[2]);
    

In [ ]:
imgs = pim.transform(dgms)

In [ ]:
imgs_array = np.array([img.flatten() for img in imgs])
X_train, X_test, y_train, y_test = train_test_split(imgs_array, labels, test_size=0.40, random_state=42)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test), set(labels)